# darknet 불러오기 (안될때 졸라많음)

In [2]:
import numpy as np
import darknet
import cv2
import os
net = darknet.load_net_custom("yolov4.cfg".encode('utf-8'), "yolov4.weights".encode('utf-8'), 0,1)
meta = darknet.load_meta("coco.data".encode('utf-8'))



# 데이터 패스 들고오기

In [3]:
import os
car_class=set()
path_jpg = []
def search(dirname):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1]
                car_name=os.path.splitext(full_filename)[0]
                if ext=='.JPG' :
                    path_jpg.append(full_filename)
                    print(full_filename)# 여기가 전체 파일
    except PermissionError:
        pass


dir_="./kcar_resolution"
search(dir_)

./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_13시대_H165_V0.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_18시대_H15_V15.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_17시대_H270_V0.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_18시대_H240_V30.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_11시대_H180_V45.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_11시대_H105_V45.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_실내_H315_V60.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_16시대_H60_V45.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_14시대_H210_V15.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_17시대_H75_V60.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_14시대_H0_V15.JPG
./kcar_resolution/Hyundai_KONA 1.6T/resoltest_현대_KONA 1.6T_2017_실내_H285_V60.JPG
./kcar_resolution/Hyundai_K

# 크롭된 사진이 들어갈 폴더 만들기

In [4]:
# path_jpg[0].split('/')[7] # 4 5 6 
car_folder = []
for i in path_jpg:
    i =i.split('/')
    car_folder.append(i[2])
    

In [5]:
# 차종 갯수는 100개 맞음
car_folder = set(car_folder)
len(car_folder)

84

In [6]:
os.mkdir('kcar_resol_crop')
# os.mkdir(os.getcwd()+'/kcar_crop/a') 연습으로 해봣음

FileExistsError: [Errno 17] File exists: 'kcar_resol_crop'

In [7]:
for i in car_folder:
    os.mkdir(os.getcwd() + '/kcar_resol_crop/'+i)

FileExistsError: [Errno 17] File exists: '/home/pirl/project/yolo4/kcar_resol_crop/Hyundai_소나타 YF'

In [12]:
path_jpg[1].split('/')

# path[4]+'_'+path[5]+'_'+path[6]


['.',
 'kcar_resolution',
 'Hyundai_KONA 1.6T',
 'resoltest_현대_KONA 1.6T_2017_18시대_H15_V15.JPG']

In [13]:
img = cv2.imread(path_jpg[14])
detections = darknet.detect_image(net, meta, img, thresh=0.2)
# len(path_jpg)
detections
# x1 = int(round((detections[0][2][0]) - (detections[0][2][2]/2.0))) # top left x1 
# y1 = int(round((detections[0][2][1]) - (detections[0][2][3]/2.0))) # top left xy 
# x2 = int(round((detections[0][2][0]) + (detections[0][2][2]/2.0))) # bottom right x2 
# y2 = int(round((detections[0][2][1]) + (detections[0][2][3]/2.0))) # bottom right y2 
# image_crop = img[y1:y2,x1:x2]   
# image_crop

# cv2.imwrite(path_jpg[51].split('/')[9],image_crop)
# path_jpg[12]


[(b'car',
  0.9781468510627747,
  (452.3213195800781,
   234.6644744873047,
   364.5447082519531,
   218.97911071777344)),
 (b'car',
  0.766907811164856,
  (442.1675109863281,
   34.6497688293457,
   61.277164459228516,
   29.186447143554688))]

In [ ]:
# 디텍트는 이런종류로함
vehicle = [b'car', b'truck']


cnt = 0
crop_cnt = 0
try_except = 0
no_crop =0 

#이제 저장을 해보자
for path in path_jpg: 
    p=path.split('/')
    crop_path = os.getcwd()+'/kcar_resol_crop/'+p[2]

    img = cv2.imread(path)  #사진 불러오기
    detections = darknet.detect_image(net, meta, img, thresh=0.4) # 디텍트 실행
    
    if detections ==[]:
        cv2.imwrite(crop_path+'/resolcrop_'+path.split('/')[3],img)
        cnt+=1
        no_crop+=1
        print('디텍안됨 남은 갯수 :'+str(cnt))
    elif detections[0][0] in vehicle: 
        size = []
        for i in detections:
            x1 = int(round((i[2][0]) - (i[2][2]/2.0))) # top left x1 
            y1 = int(round((i[2][1]) - (i[2][3]/2.0))) # top left xy 
            x2 = int(round((i[2][0]) + (i[2][2]/2.0))) # bottom right x2 
            y2 = int(round((i[2][1]) + (i[2][3]/2.0))) # bottom right y2 
            garo = y2-y1
            sero = x2-x1
            size.append(garo*sero)
        max_size = max(size)
        detection_index = size.index(max_size)
        
        x1 = int(round((detections[detection_index][2][0]) - (detections[detection_index][2][2]/2.0))) # top left x1 
        y1 = int(round((detections[detection_index][2][1]) - (detections[detection_index][2][3]/2.0))) # top left xy 
        x2 = int(round((detections[detection_index][2][0]) + (detections[detection_index][2][2]/2.0))) # bottom right x2 
        y2 = int(round((detections[detection_index][2][1]) + (detections[detection_index][2][3]/2.0))) # bottom right y2 
        image_crop = img[y1:y2,x1:x2]      
        
        try:
            cv2.imwrite(crop_path+'/resolcrop_'+path.split('/')[3],image_crop)
            cnt+=1
            crop_cnt+=1
            print('크롭됨 남은 갯수 :'+str(cnt))
        except:
            cv2.imwrite(crop_path+'/resolcrop_'+path.split('/')[3],img)
            cnt+=1
            try_except+=1
            print('예외 처리됨 :'+str(cnt))

    else : 
        cv2.imwrite(crop_path+'/resolcrop_'+path.split('/')[3],img)
        cnt+=1
        no_crop+=1
        print('크롭안됨 남은 갯수 :'+str(cnt))

    

크롭됨 남은 갯수 :1
크롭됨 남은 갯수 :2
크롭됨 남은 갯수 :3
크롭됨 남은 갯수 :4
디텍안됨 남은 갯수 :5
크롭됨 남은 갯수 :6
크롭됨 남은 갯수 :7
크롭됨 남은 갯수 :8
크롭됨 남은 갯수 :9
크롭됨 남은 갯수 :10
디텍안됨 남은 갯수 :11
크롭안됨 남은 갯수 :12
크롭안됨 남은 갯수 :13
크롭됨 남은 갯수 :14
크롭됨 남은 갯수 :15
크롭됨 남은 갯수 :16
크롭안됨 남은 갯수 :17
크롭됨 남은 갯수 :18
크롭안됨 남은 갯수 :19
디텍안됨 남은 갯수 :20
크롭됨 남은 갯수 :21
크롭됨 남은 갯수 :22
크롭됨 남은 갯수 :23
크롭됨 남은 갯수 :24
크롭됨 남은 갯수 :25
크롭됨 남은 갯수 :26
크롭됨 남은 갯수 :27
크롭됨 남은 갯수 :28
크롭됨 남은 갯수 :29
크롭됨 남은 갯수 :30
크롭됨 남은 갯수 :31
크롭됨 남은 갯수 :32
크롭됨 남은 갯수 :33
디텍안됨 남은 갯수 :34
크롭됨 남은 갯수 :35
크롭됨 남은 갯수 :36
크롭됨 남은 갯수 :37
크롭됨 남은 갯수 :38
크롭됨 남은 갯수 :39
디텍안됨 남은 갯수 :40
크롭됨 남은 갯수 :41
크롭됨 남은 갯수 :42
디텍안됨 남은 갯수 :43
크롭됨 남은 갯수 :44
크롭됨 남은 갯수 :45
크롭됨 남은 갯수 :46
크롭됨 남은 갯수 :47
크롭됨 남은 갯수 :48
크롭됨 남은 갯수 :49
디텍안됨 남은 갯수 :50
크롭됨 남은 갯수 :51
크롭됨 남은 갯수 :52
크롭됨 남은 갯수 :53
크롭됨 남은 갯수 :54
크롭됨 남은 갯수 :55
크롭됨 남은 갯수 :56
디텍안됨 남은 갯수 :57
디텍안됨 남은 갯수 :58
크롭됨 남은 갯수 :59
크롭됨 남은 갯수 :60
크롭됨 남은 갯수 :61
크롭안됨 남은 갯수 :62
크롭됨 남은 갯수 :63
크롭됨 남은 갯수 :64
크롭됨 남은 갯수 :65
디텍안됨 남은 갯수 :66
크롭됨 남은 갯수 :67
크롭됨 남은 갯수 :68
크롭됨 남은 갯수 :69
크롭됨 남은 갯수 :70
크롭안됨 남은 갯수 :71

크롭됨 남은 갯수 :549
크롭됨 남은 갯수 :550
크롭됨 남은 갯수 :551
크롭됨 남은 갯수 :552
크롭됨 남은 갯수 :553
크롭됨 남은 갯수 :554
크롭됨 남은 갯수 :555
크롭됨 남은 갯수 :556
크롭됨 남은 갯수 :557
디텍안됨 남은 갯수 :558
디텍안됨 남은 갯수 :559
크롭됨 남은 갯수 :560
크롭됨 남은 갯수 :561
디텍안됨 남은 갯수 :562
크롭됨 남은 갯수 :563
크롭됨 남은 갯수 :564
크롭됨 남은 갯수 :565
크롭됨 남은 갯수 :566
크롭됨 남은 갯수 :567
크롭됨 남은 갯수 :568
디텍안됨 남은 갯수 :569
크롭됨 남은 갯수 :570
크롭안됨 남은 갯수 :571
크롭됨 남은 갯수 :572
크롭됨 남은 갯수 :573
크롭됨 남은 갯수 :574
디텍안됨 남은 갯수 :575
크롭안됨 남은 갯수 :576
크롭됨 남은 갯수 :577
크롭됨 남은 갯수 :578
크롭됨 남은 갯수 :579
크롭됨 남은 갯수 :580
크롭됨 남은 갯수 :581
크롭됨 남은 갯수 :582
크롭됨 남은 갯수 :583
크롭됨 남은 갯수 :584
디텍안됨 남은 갯수 :585
크롭됨 남은 갯수 :586
디텍안됨 남은 갯수 :587
크롭됨 남은 갯수 :588
크롭됨 남은 갯수 :589
크롭됨 남은 갯수 :590
크롭안됨 남은 갯수 :591
크롭됨 남은 갯수 :592
디텍안됨 남은 갯수 :593
크롭됨 남은 갯수 :594
크롭됨 남은 갯수 :595
크롭됨 남은 갯수 :596
크롭됨 남은 갯수 :597
크롭됨 남은 갯수 :598
디텍안됨 남은 갯수 :599
크롭됨 남은 갯수 :600
크롭됨 남은 갯수 :601
크롭됨 남은 갯수 :602
크롭됨 남은 갯수 :603
크롭됨 남은 갯수 :604
디텍안됨 남은 갯수 :605
크롭됨 남은 갯수 :606
크롭됨 남은 갯수 :607
크롭됨 남은 갯수 :608
크롭됨 남은 갯수 :609
크롭됨 남은 갯수 :610
크롭됨 남은 갯수 :611
크롭됨 남은 갯수 :612
크롭됨 남은 갯수 :613
크롭됨 남은 갯수 :6

In [15]:
print("크롭된갯수:"+str(crop_cnt))
print("예외처리된 갯수:"+str(try_except))
print("크롭안된갯수:"+str(no_crop))
 
 

크롭된갯수:791
예외처리된 갯수:0
크롭안된갯수:222
